# Analysis Notebooks for Thesis.

Supports KMeans and DBSCAN. Also performs preliminary tests for optimal algorithm parameters.

### Imports and Preamble

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from typing import List, Dict, Tuple, Union 
from functools import reduce

import numpy as np
import pandas as pd
# import jax
# import jax.numpy as jnp
# import math
# from tqdm import tqdm
import pickle
import json

import py3Dmol
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from moleculib.protein.datum import ProteinDatum
from moleculib.graphics.py3Dmol import plot_py3dmol, plot_py3dmol_grid

from moleculib.protein.transform import (
    ProteinCrop,
    TokenizeSequenceBoundaries,
    ProteinPad,
    MaybeMirror,
    BackboneOnly,
    DescribeChemistry
)

from helpers.database import populate_representations, get_column, get_scalars, whatis
from helpers.edges import connect_edges, CascadingEdges
from helpers.cascades import Cascade, MakeCascade, Metrics, MetricsPair, MakeMetricsPair
from helpers.load_data import LoadData, save_df, save_edges

In [ ]:
FOLDER = "data/denim-energy-1008-embeddings"
embeddings_file = "encoded_dataset.pkl"
sliced_proteins_file = "sliced_dataset.pkl"
tsne_file = "encoded_dataset_tsne.json"

# Load data from files
dataloader = LoadData(FOLDER, embeddings_file, sliced_proteins_file, tsne_file)
dataloader.load_all()

# Make objects
reps, _ = populate_representations(dataloader.encoded_dataset, 
                                   dataloader.sliced_dataset, 
                                   dataloader.tsne_dataset)
df = reps.to_dataframe()
print(f"Loaded full dataset: {df.shape}")

# Calculate the number of nodes per level
nodes_per_level = df.groupby('level').size()
print("Number of nodes per level:")
display(nodes_per_level)

# Count the "None" datums
n_none_datums = df[df['datum'].isnull()].shape[0]
print(f"Number of None datums: {n_none_datums}")

We need to do data cleaning as follows: Drop the none datums. Then every key in the edges bottom up dictionary that does not cascade to the top, is dropped from the dataframe (including that entire pdb id). 

In [ ]:
# Process for clustering algos...

df_sample = df.dropna(subset=['datum']).reset_index(drop=True)  # drop nans
df_sample = df_sample[df_sample['level'] != 0].reset_index(drop=True)  # drop level 0

print(f"Shape of sample after drops: {df_sample.shape}")

# Compute edges
kernel, stride = 5, 2
edges_top_down, edges_bottom_up, n_misses = connect_edges(df_sample, kernel, stride)
make_cascades = CascadingEdges(edges_bottom_up)
print(f"Misses: {n_misses}")
whatis(edges_top_down, edges_bottom_up, make_cascades)


In [ ]:
SAVE = False
if SAVE:

    ABS_PATH_TO_VIZ = "/Users/moniradev/Documents/projects/foreign/picture-picture/public/data"

    # Drop scalars and datums for visualization
    save_df(df_sample.drop(columns=['scalar_rep', 'datum']), "df_final", folder=ABS_PATH_TO_VIZ)
    save_edges(edges_bottom_up, "edges_bottom_up_final", folder=ABS_PATH_TO_VIZ)



In [ ]:
# Filter the dataframe based on both pdb_id and level
# 1dabA
filtered_df = df_sample[(df_sample['pdb_id'] == '1dbvnA') & (df_sample['level'] == 2)]
# display(filtered_df)
print(filtered_df.index.tolist())


In [ ]:
from helpers.cascades import MakeMetricsPair
# u, v = 176533, 200852

# u, v = 188260, 194176

u, v = 203599, 180218


# u, v = 187047, 150546

# u, v = 150633, 187126

# u, v = 48430, 48431

# u, v = 204856, 197267

# u, v = 202778, 197051

# u, v = 201935, 1109   

# ubi = "MQIFVKTLTG KTITLEVEPS DTIENVKAKI QDKEGIPPDQ QRLIFAGKQL EDGRTLSDYN IQKESTLHLV LRLRGG"
ubiquitin_scaffold = "MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG"
# MQIFVKTLT-[Motif]-GKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG

def scaffolded_motif(motif, scaffold=ubiquitin_scaffold):
    return f"{scaffold[:9]}{motif}{scaffold[9:]}"


# Position 8 and 9


us, vs = make_cascades(u), make_cascades(v)
print(us, vs)
metrics_pair = MakeMetricsPair(df_sample, us, vs)()
# cas1, cas2 = metrics_pair

In [ ]:
from helpers.utils import d
import random

def generate_random_aa_sequence(length=13):
    letters = list(filter(lambda x: x not in ['UNK', 'MASK', 'PAD'], d.values()))
    return ''.join(random.choices(letters, k=length))

random_sequence = generate_random_aa_sequence()
print("Random 13-letter amino acid sequence:", random_sequence)
print("Random scaffold using this raandom motif:", scaffolded_motif(random_sequence))


In [ ]:
view1, view2, indices1, indices2 = metrics_pair.plot_cascade_pair(
    metrics_pair.cascade1, metrics_pair.cascade2, return_indices=True
)
print(f"Indices1: {indices1}")
print(f"Indices2: {indices2}")
view1.show()
view2.show()


In [ ]:
class SavePDB:
    """Given a dataframe of encodings, save .pdb files of datums specified by index."""
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def __call__(self, u: int, filename: str = "", folder: str = "saved_pdbs/"):
        """Given an index, or list of indices in the dataframe, save the corresponding PDB data as a .pdb file.
            The naming scheme is `index.pdb` where index is the index of the datum in the dataframe.
        """
        if isinstance(u, list):
            for index in u:
                pdb_data = self.df.iloc[index]['datum'].to_pdb_str()
                self.save_as_pdb(pdb_data, f"{filename+str(u)}", folder)
        else:
            pdb_data = self.df.iloc[u]['datum'].to_pdb_str()
            self.save_as_pdb(pdb_data, f"{filename+str(u)}", folder)

    @staticmethod
    def save_as_pdb(pdb_data: str, filename: str, folder: str = "saved_pdbs/"):
        with open(f"{folder}/{filename}.pdb", "w") as file:
            file.write(pdb_data)

saved_pdb = SavePDB(df_sample)
saved_pdb(u)

df_sample.iloc[[u,v]]['datum'].values[0].to_pdb_str()

In [ ]:
print(f"Results for ids: {u}, {v}")
print(metrics_pair)
print("First column sequences (short): ")
short_seq1 = aa_map(metrics_pair.cascade1.sequences[0])[0]
short_seq2 = aa_map(metrics_pair.cascade2.sequences[0])[0]
print(short_seq1)
print(short_seq2)
print("Scaffolded motifs:")
print(scaffolded_motif(short_seq1))
print(scaffolded_motif(short_seq2))

In [ ]:
from tmtools import tm_align

coords1 = metrics_pair.cascade1.datums[0].atom_coord[:, 1, :].reshape(13, 3)
coords2 = metrics_pair.cascade2.datums[0].atom_coord[:, 1, :].reshape(13, 3)
seq1 = aa_map(metrics_pair.cascade1.sequences[0])[0]
seq2 = aa_map(metrics_pair.cascade2.sequences[0])[0]
whatis(coords1, coords2, seq1, seq2)

print()
print(seq1)
print(seq2)
res = tm_align(coords1, coords2, 'A'*13, 'A'*13)
print(res.t, res.u)

print(res.tm_norm_chain1)
print(res.tm_norm_chain2)


In [ ]:
cd20s = ['6PE9', '6TKB', '6PE8', '6TKF', '6TKE', '6TKD', '6TKC', '1QSC', '6BRB', '3LKJ',
         '6PE7', '1ALY']
ms_related = ['6H24', '1PY9', '5HIU', '6FG1', '6FG2', '4Q6R', '4GMV']

In [ ]:

from helpers.utils import CheckPDBs

checker = CheckPDBs(cd20s + ms_related)
# common_keys, missed_keys = checker.ds(chroma_full.encoded_dataset)
common_keys, missed_keys = checker.df(df_sample)

# Print the PDB IDs not in common
# print(f"Length of request pdb ids: {len(pdbids)}, length of encoded keys: {len(chroma_full.encoded_dataset.keys())}")
print(f"There are {len(missed_keys)} missed keys and {len(common_keys)} common keys")
print()
print("Missed pdb ids:")
print(missed_keys)

In [ ]:
# Filter the dataframe to get rows where the 'pdb_id' contains '1f00'
filtered_df = df_sample[df_sample['pdb_id'].str.contains('1bhx')]
print(filtered_df['pdb_id'].unique())
df_sample['pdb_id'].unique()



In [ ]:
from moleculib.assembly.datum import AssemblyDatum

# Given a list of PDB ids, pull them from moleculib and visualize

max_chain_len = 253  # max length for denim-energy model
protein_transform = [
    ProteinCrop(crop_size=max_chain_len),
    TokenizeSequenceBoundaries(),
    MaybeMirror(hand='left'),
    ProteinPad(pad_size=max_chain_len, random_position=False),
    BackboneOnly(filter=True),
    DescribeChemistry(),
]

def transform(datum):
    return reduce(lambda x, f: f.transform(x), protein_transform, datum)

class FetchPDBids:
    """Fetch PDB ids as AssemblyDatums."""
    def __init__(self, pdb_ids: List[str]):
        self.pdb_ids = [pdb_id.lower() for pdb_id in pdb_ids]
        self.datums = []
        self.transformed = []  # list of transformed ProteinDatums

    def __call__(self):
        print(f"Fetching {len(self.pdb_ids)} PDB IDs...", end=" ")
        for pdb_id in self.pdb_ids:
            assembly = AssemblyDatum.fetch_pdb_id(pdb_id,)
            for datum in assembly.protein_data:
                # datum.idcode = pdb_id
                self.datums.append(datum)
            print(f"{pdb_id}, ", end="")
        print("\nDone")

    def togrid(self, k=None, num_columns=3, use_transformed=False):
        if k is None:
            k = len(self.datums)
        if use_transformed:
            if self.transformed == []:
                self.transform()
            datum_grid = self.make_grid(self.transformed[:k], num_columns)
        else:
            datum_grid = self.make_grid(self.datums[:k], num_columns)
        return datum_grid
    
    @staticmethod
    def make_grid(datums: List[ProteinDatum], num_columns=3):
        return [datums[i:i + num_columns] for i in range(0, len(datums), num_columns)]
    
    def transform(self):
        self.transformed = [transform(datum) for datum in self.datums]

fetcher = FetchPDBids(master_list_unique)
fetcher()
fetcher.transform()
print(f"Number of fetched datums: {len(fetcher.datums)}")

In [ ]:
[d.idcode for d in fetcher.transformed]

In [ ]:
# Get list index of "2KL8" from missed_keys

term_pair = ['2kl8', '2ln3']
term_pair_datums = [fetcher.datums[fetcher.pdb_ids.index(pdb_id)] for pdb_id in term_pair]
print(term_pair_datums)
fetcher.make_grid(term_pair_datums, num_columns=2)
plot_py3dmol_grid(fetcher.make_grid(term_pair_datums, num_columns=2),)

In [ ]:
len(term_pair_datums[0])

In [ ]:
plot_py3dmol_grid(fetcher.togrid(k=15, num_columns=3, use_transformed=True))

In [ ]:
from tmtools import tm_align
from helpers.utils import DistanceMapMetric, DistanceSeqMetric


def calphas_from_datum(datum):
    return datum.atom_coord[:, 1, :].reshape(-1, 3)

coords1 = calphas_from_datum(term_pair_datums[0])
coords2 = calphas_from_datum(term_pair_datums[1])
print(coords1.shape, coords2.shape)

res = tm_align(coords1, coords2, 'A'*85, 'A'*83)
print(res.t, res.u)

print(res.tm_norm_chain1)
print(res.tm_norm_chain2)


In [ ]:
print(DistanceMapMetric()(term_pair_datums[0], term_pair_datums[1]))
DistanceSeqMetric()(term_pair_datums[0], term_pair_datums[1])

In [ ]:

# Split the list of fetched datums into a grid with at most 3 columns
num_columns = 3
datum_grid = fetcher.togrid(num_columns, use_transformed=True)
print(datum_grid)

plot_py3dmol_grid(datum_grid)
# common_.plot(view=py3Dmol.view())

In [ ]:
class Idx2Datum:
    def __init__(self, df):
        self.df = df

    def __call__(self, *idxs):
        return self.df.loc[idxs, 'datum'].values

In [ ]:
from helpers.statistics import ComputeDistanceMatrix, RadiusNeighbors, sort_distance_graph

distMx = ComputeDistanceMatrix(df_sample)

def analyze_distance_matrix(level):
    # Compute the distance matrix at the specified level
    matrix = distMx(level=level)

    # Convert the csr matrix to a dense format to handle it with numpy
    dense_matrix = matrix.toarray()

    # Flatten the matrix to get the distribution of distance values
    flat_distances = dense_matrix.flatten()

    # Calculate quantiles
    quantiles = np.quantile(flat_distances, [0.25, 0.5, 0.75])

    # Print quantile information
    print("Quantiles of distance values:")
    print(f"25th percentile: {quantiles[0]}")
    print(f"50th percentile (median): {quantiles[1]}")
    print(f"75th percentile: {quantiles[2]}")

    # Calculate and print distribution details
    print("Distribution details of distance values:")
    print(f"Minimum distance: {flat_distances.min()}")
    print(f"Maximum distance: {flat_distances.max()}")
    print(f"Mean distance: {flat_distances.mean()}")
    print(f"Standard deviation: {flat_distances.std()}")

# Example usage
analyze_distance_matrix(level=3)


In [ ]:
from helpers.utils import aa_map
aa_map(MakeCascade.datum_to_sequence(metrics_pair.cascade1.datums[-1]))



In [ ]:
import random

class CascadeDropper:
    def __init__(self, df, edges_bottom_up):
        self.df = df
        self.edges_bottom_up = edges_bottom_up

    def drop_random_cascades(self):
        # Iterate through the bottom indices
        bottom_indices = self.df[self.df['level'] == 1].index
        # bottom_indices = [idx for idx in self.edges_bottom_up if self.df.loc[idx, 'level'] == 1]

        # Initialize list to collect cascades to drop
        cascades_to_drop = []

        # Get cascades and randomly drop 20% of them
        for idx in bottom_indices:
            if random.random() < 0.20:  # 20% chance to drop this cascade
                cascade = make_cascades(idx, verbose=False)
                cascades_to_drop.extend(cascade)

        # Collect indices from the dropped cascades
        indices_to_drop_from_cascades = list(set(cascades_to_drop))

        # Drop these indices from the dataframe
        self.df = self.df.drop(indices_to_drop_from_cascades)
        print(f"Shape of dataframe after dropping 20% of cascades: {self.df.shape}")
        return self.df

# Instantiate and use the CascadeDropper
cascade_dropper = CascadeDropper(df_sample, edges_bottom_up)
df_sample_clean = cascade_dropper.drop_random_cascades()



Basic mutation is sequential, but is there a way in which biology conserves structure as well?

10^16 13-mers. 

Is there a case where something is similar in structure space and dissimilar in sequence space? 

<ol>
<li>Find pairs similar in ophiuchus latent space that map to divergent pairs in higher levels (in ophiuchus space)</li>
<li>Of the 111k 5-mer vector space embeddings, what is the subset of those vectors which </li>
</ol>


The parts that are closer at the lower level and appear in things very different suggests of big mutation (or big difference). 

1. For a specific level: find dmin, then list out all the pairs that are 10 x dmin.


The model provides an alignment mechanism in latent space ("Multiple Latent Alignment")


Thesis is the idea of building blocks. Why are we so sure that biology has building blocks? The reason is that biology builds 300-mer proteins and it cnanot be done from scratch. 

A helix is an examples of a structural motif 

A helix can have lots  of amino acids, but it must present a correct seq of amino acids to build a helix helxi (ie building locko). There are specific locations that need to match to build a building block. The same exact sequecne is too over constarining. The right building block is the right number f amino acids in the right place. 


<i>Small number of amino acids in the right place, that interact with another set of blocks that are a small number of amino acids in the right place.</i>

If we take things close in level 1 and swap them out using AlphaFold, what does that build? 

But then isn't a beta helix in some sense exactly this? 

Chapters:

Chapter 1:

level 1 pairs that are close, map to level 4 pairs that are broad. Call those "building blocks".  What is the intellectual insight of a building block? 

Chapter 2: 

<b>If I have time:</b> AlphaFold. Biology would have taken a part, mutated a point mutation, and ended up with the same structure. THen biology would have to do a crossover mutation and reuse it somewhere else. 

In [ ]:

idx = 202409
# df_sample.iloc[202409]
df[df['pdb_id'] == '1b2wL']

In [ ]:
from helpers.statistics import ComputeDistanceMatrix, RadiusNeighbors, sort_distance_graph

distMx = ComputeDistanceMatrix(df_sample)
matrix = distMx(level=3)

In [ ]:
# Pick sample candidates
main_df = df_small.dropna(subset=['datum'])

u, v = 1342, 3834

display(main_df.loc[[u, v]])
us, vs = make_cascades(u), make_cascades(v)
print(us, vs)



MakeCascade.plot_datums(Idx2Datum(df_small)(us)).show()

